This notebook uses distilgpt2 model and its tokenizer.
gpt2 is multilingular model which is also trained on Armenian.
The usage of weights only with fine tuning will not give us promissing results, because it is also trained on a lot of languages and all weights of the Transformer are optimized for all cases. So, by fine tuning we are trying to use armenian model with model which is trained on english, french ... which gives unlogical result. So we had to train it on Armenian seperately, without fine tuning. 


In [34]:
from transformers import TFGPT2LMHeadModel,GPT2Tokenizer
import tensorflow as tf

# loading model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = TFGPT2LMHeadModel.from_pretrained("distilgpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [35]:
# Scraped data from wikipedia with the 35000 sentences
file_path = '../../Data/Scraped/wiki_data_3.txt'

with open(file_path, encoding="utf-8") as f:
    text = f.read()

tokenized_text = tokenizer.encode(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (7813227 > 1024). Running this sequence through the model will result in indexing errors


In [36]:
#max length of input
examples = []
block_size = 100
for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
    examples.append(tokenized_text[i:i + block_size])


In [37]:
# with length 100 each input has 99 length and label 1
inputs, labels = [], []
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])

# use tensorflow packages to run on GPU
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))

BATCH_SIZE = 12
BUFFER_SIZE = 10000

# increase entropy by shuffling data
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# setting optimizer for NN
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)

# setting loss which should be minimized, we ues SparseCategoricalCrossentropy which handles dummies by itself
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metric which we want to visualize
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# adding all to compile
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [38]:
# train 10 epochs and save all weights
for ep in range(1,10):
    model.fit(dataset, epochs=1)
    model.save_weights("mymodel_epoch_" + str(ep))

6511/6511 [==============================] - 2655s 408ms/step - loss: 0.7181 - logits_loss: 0.7181 - logits_accuracy: 0.7653 - past_key_values_1_accuracy: 7.2861e-05 - past_key_values_2_accuracy: 6.1888e-05 - past_key_values_3_accuracy: 5.6351e-05 - past_key_values_4_accuracy: 6.6623e-05 - past_key_values_5_accuracy: 9.3923e-05 - past_key_values_6_accuracy: 5.4923e-05


In [39]:
model.save_weights("finalmodel")

In [40]:
model.load_weights("finalmodel")


In [67]:
text = "Ես գնացի "

In [68]:
input_ids = tokenizer.encode(text,return_tensors = 'tf')

In [69]:
greedy_output = model.generate(input_ids,
                               max_length = 50,
                               num_beams = 5,
                               early_stopping = True,
                               no_repeat_ngram_size = 5,
                               )

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [70]:

print(tokenizer.decode(greedy_output[0],skip_special_tokens = True))

Ես գնացի առաջնության համար 
